In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\acer\\OneDrive\\Desktop\\VIT MSD\\Project\\End to End ML ops along with deployment\\End-To-End-Machine-Learning-and-ML-flow-Project'

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tushargoutam"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "b7cfe0b4e3d655802d2fe95cd35fd161485991a5"

In [3]:
import dagshub
dagshub.init(repo_owner='tushargoutam',
             repo_name='End-To-End-Machine-Learning-and-ML-flow-Project',
             mlflow=True)

Accessing as tushargoutam

Initialized MLflow to track repo "tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project"

Repository tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project initialized!

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [5]:
from src.Project.constants import *
from src.Project.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri = "https://dagshub.com/tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project.mlflow"
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

c:\Users\acer\OneDrive\Desktop\VIT MSD\Project\End to End ML ops along with deployment\End-To-End-Machine-Learning-and-ML-flow-Project\mlops\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local JSON file
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, artifact_path="model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, artifact_path="model")


In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-19 21:34:49,836 : INFO : common : yaml file : config\config.yaml loaded sucessfully]
[2025-10-19 21:34:49,838 : INFO : common : yaml file : params.yaml loaded sucessfully]
[2025-10-19 21:34:49,840 : INFO : common : yaml file : schema.yaml loaded sucessfully]
[2025-10-19 21:34:49,841 : INFO : common : Created directory at : artifacts]
[2025-10-19 21:34:49,841 : INFO : common : Created directory at : artifacts/model_evaluation]
[2025-10-19 21:34:51,190 : INFO : common : json file saved at : artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/10/19 21:35:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.
